In [ ]:
import pandas as pd
import numpy as np
import requests
from owslib.ogcapi.features import Features
from datetime import datetime, timedelta
# Initialize the Features API
oafeat = Features("https://api.weather.gc.ca/")

def create_date_range(start_date, end_date):
    return pd.date_range(start=start_date, end=end_date)

def extract_flow_data_us(station_list, start_date, end_date):
    dates = create_date_range(start_date, end_date)
    empty_df = pd.DataFrame({'Date': dates})
    station_info = []

    for station in station_list:
        empty_df[station] = np.nan
    
    for station in station_list:
        url = f"https://waterservices.usgs.gov/nwis/dv/?format=json&sites={station}&startDT={start_date}&endDT={end_date}&parameterCd=00060&statCd=00003"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if 'timeSeries' in data['value']:
                records = data['value']['timeSeries'][0]['values'][0]['value']
                flow_data = pd.DataFrame(records)
                flow_data['value'] = pd.to_numeric(flow_data['value'], errors='coerce')
                flow_data['dateTime'] = pd.to_datetime(flow_data['dateTime']).dt.date
                flow_data['dateTime'] = flow_data['dateTime'].astype(str)
                empty_df['Date'] = empty_df['Date'].astype(str)
                for date, flow in zip(flow_data['dateTime'], flow_data['value']):
                    empty_df.loc[empty_df['Date'] == date, station] = flow
                
                # Collect station information
                site_info = data['value']['timeSeries'][0]['sourceInfo']
                station_info.append({
                    'Station_Number': site_info['siteCode'][0]['value'],
                    'Station_Name': site_info['siteName'],
                    'Latitude': site_info['geoLocation']['geogLocation']['latitude'],
                    'Longitude': site_info['geoLocation']['geogLocation']['longitude'],
                    'Drainage_Area': next((prop['value'] for prop in site_info['siteProperty'] if prop['name'] == 'drain_area_va'), None)
                })
            else:
                print(f"Flow data column not found for station: {station}")
        else:
            print(f"Failed to retrieve data for station: {station}")
    
    return empty_df, station_info

def fetch_hydrometric_data_ca(station_numbers, start_date, end_date, limit=500):
    combined_data = pd.DataFrame({'Date': create_date_range(start_date, end_date)})
    station_info = []

    for station_number in station_numbers:
        offset = 0
        full_data = []

        while True:
            url = f"https://api.weather.gc.ca/collections/hydrometric-daily-mean/items"
            params = {
                'STATION_NUMBER': station_number,
                'datetime': f"{start_date}/{end_date}",
                'limit': limit,
                'offset': offset,
                'f': 'json'
            }

            response = requests.get(url, params=params)
            response_data = response.json()

            if 'features' in response_data and response_data['features']:
                full_data.extend(response_data['features'])
                offset += limit
                if len(response_data['features']) < limit:
                    break
            else:
                break

        if full_data:
            # Create data_list with dates and discharge values
            data_list = [
                {
                    'Date': feature['properties']['DATE'],
                    'value': feature['properties']['DISCHARGE'] if feature['properties']['DISCHARGE'] is not None else -1000
                }
                for feature in full_data
            ]

            # Convert data_list to DataFrame
            flow_data = pd.DataFrame(data_list)

            # Convert 'value' to numeric and 'Date' to datetime.date
            flow_data['value'] = pd.to_numeric(flow_data['value'], errors='coerce')
            flow_data['Date'] = pd.to_datetime(flow_data['Date']).dt.date
            flow_data['Date'] = flow_data['Date'].astype(str)
            combined_data['Date'] = combined_data['Date'].astype(str)

            # Create a new column for the station in combined_data
            combined_data[station_number] = np.nan

            # Populate combined_data with flow data
            for date, flow in zip(flow_data['Date'], flow_data['value']):
                combined_data.loc[combined_data['Date'] == date, station_number] = flow

            # Collect station information
            first_feature = full_data[0]['properties']
            geometry = full_data[0]['geometry']
            station_info.append({
                'Station_Number': first_feature['STATION_NUMBER'],
                'Station_Name': first_feature['STATION_NAME'],
                'Latitude': geometry['coordinates'][1],
                'Longitude': geometry['coordinates'][0],
                'Drainage_Area': first_feature.get('DRAINAGE_AREA_GROSS', None)
            })
        else:
            print(f"Flow data not found for station: {station_number}")

    return combined_data, station_info

def write_flow_data_to_file_obstxt(file_path, flow_data, site_details):
    # Replace NaN with -1.000
    flow_data = flow_data.fillna(-1.000)
    
    # Convert Date column to datetime
    flow_data['Date'] = pd.to_datetime(flow_data['Date'])

    # Prepare the text file
    with open(file_path, "w") as file_conn:
        # Write header line
        start_date = flow_data['Date'].min()
        end_date = flow_data['Date'].max()
        file_conn.write(f"Observedstreamflow\t{start_date.strftime('%Y/%m/%d')}\t{end_date.strftime('%Y/%m/%d')}\n")
        
        # Write second line
        num_stations = flow_data.shape[1] - 1
        num_days = flow_data.shape[0]
        start_year = start_date.strftime('%Y')
        start_day_of_year = start_date.timetuple().tm_yday
        file_conn.write(f"{num_stations}  {num_days}  {num_days}  24 {start_year}  {start_day_of_year} 00\n")
        
        # Write station metadata with integer lat*60 and lon*60
        for station_id in flow_data.columns[1:]:
            station_info = next((item for item in site_details if item["Station_Number"] == station_id), None)
            if station_info:
                lat = station_info['Latitude']
                lon = station_info['Longitude']
                drainage_area = station_info['Drainage_Area']
                if drainage_area is None:
                    drainage_area = -1.0
                station_name = station_info['Station_Name']
                file_conn.write(f"{int(lat * 60):4d} {int(lon * 60):4d} {lat:12.6f} {lon:12.6f} {station_id:12s} {float(drainage_area):12.3f} {station_name}\n")
        
        # Write flow data with 12.4 decimal format
        for i in range(num_days):
            flow_values = flow_data.iloc[i, 1:].values  # Exclude the Date column
            formatted_flow_values = " ".join(f"{x:12.4f}" for x in flow_values)
            file_conn.write(f"{formatted_flow_values}\n")
            
def write_flow_data_to_file_ensim(file_path, flow_data, site_details):
    # Replace NaN with -1.000
    flow_data = flow_data.fillna(-1.000)
    
    # Define header components
    header = [
        "########################################",
        ":FileType tb0  ASCII  EnSim 1.0",
        "#",
        "# DataType               Time Series",
        "#",
        ":Application             EnSimHydrologic",
        ":Version                 2.1.23",
        ":WrittenBy          PythonScript",
        f":CreationDate       {datetime.now().strftime('%Y-%m-%d')}",
        "#",
        "#---------------------------------------",
        ":SourceFile                   flow_data",
        "#",
        ":Name               streamflow",
        "#",
        ":Projection         LATLONG",
        ":Ellipsoid          WGS84",
        "#",
        f":StartTime          {flow_data['Date'].iloc[0]} 00:00:00.00000",
        "#",
        ":AttributeUnits            1.0000000",
        ":DeltaT               24",
        ":RoutingDeltaT         1",
        "#",
        ":ColumnMetaData",
        f"   :ColumnUnits             {' '.join(['m3/s' for _ in range(flow_data.shape[1] - 1)])}",
        f"   :ColumnType             {' '.join(['float' for _ in range(flow_data.shape[1] - 1)])}",
        f"   :ColumnName           {' '.join(flow_data.columns[1:])}",
        "   :ColumnLocationX    " + ' '.join([f"{site['Longitude'] * 60:.6f}" for site in site_details]),
        "   :ColumnLocationY    " + ' '.join([f"{site['Latitude'] * 60:.6f}" for site in site_details]),
        f"   :coeff1            {' '.join(['0.0000E+00' for _ in range(flow_data.shape[1] - 1)])}",
        f"   :coeff2            {' '.join(['0.0000E+00' for _ in range(flow_data.shape[1] - 1)])}",
        f"   :coeff3            {' '.join(['0.0000E+00' for _ in range(flow_data.shape[1] - 1)])}",
        f"   :coeff4            {' '.join(['0.0000E+00' for _ in range(flow_data.shape[1] - 1)])}",
        f"   :Value1            {' '.join(['1' for _ in range(flow_data.shape[1] - 1)])}",
        ":EndColumnMetaData",
        ":endHeader"
    ]
    
    # Write header to file
    with open(file_path, "w") as file_conn:
        file_conn.write("\n".join(header) + "\n")
        
        # Write data lines, ensuring 24-space padding and equally spaced columns (12.4 format)
        for _, row in flow_data.iterrows():
            flows = row[1:].values
            flow_string = " ".join(f"{flow:12.4f}" for flow in flows)
            file_conn.write(f"{' ' * 22}{flow_string}\n")
            
station_ca = ["05AA004", "05AA008", "05AA011", "05AA022", "05AA024", "05AA027", "05AA028", "05AA035", "05AB005", "05AB021",
         "05AB029", "05AB041", "05AC003", "05AC012", "05AC023", "05AC030", "05AC031", "05AD003", "05AD005", "05AD007",
         "05AD010", "05AD028", "05AD035", "05AD041", "05AE002", "05AE005", "05AE006", "05AE016", "05AE027", "05AE043",
         "05AG003", "05AG006", "05AH005", "05AH037", "05AH041", "05AH052", "05AH053", "05AJ001", "05BA001", "05BA002",
         "05BB001", "05BC001", "05BG006", "05BG010", "05BH004", "05BH005", "05BH014", "05BH015", "05BH016", "05BJ001",
         "05BJ004", "05BJ010", "05BK001", "05BL004", "05BL007", "05BL012", "05BL013", "05BL014", "05BL019", "05BL022",
         "05BL023", "05BL024", "05BL027", "05BM002", "05BM004", "05BM007", "05BM008", "05BM014", "05BN002", "05BN006",
         "05BN012", "05CA002", "05CA004", "05CA009", "05CA011", "05CA012", "05CB001", "05CB002", "05CB004", "05CB007",
         "05CC001", "05CC002", "05CC007", "05CC008", "05CC009", "05CC011", "05CC013", "05CD006", "05CD007", "05CE001",
         "05CE002", "05CE005", "05CE006", "05CE007", "05CE012", "05CE018", "05CE020", "05CG004", "05CG006", "05CH007",
         "05CH008", "05CH011", "05CJ006", "05CJ012", "05CK001", "05CK004", "05CK005", "05CK006", "05DA009", "05DB002",
         "05DB006", "05DC001", "05DC006", "05DD009", "05DF001", "05EA005", "05EF001", "05FA001", "05FA011", "05FE004",
         "05GG001", "05HD036", "05HD039", "05HG001", "05JE006", "05JF001", "05JF005", "05JK002", "05JK007", "05JM001",
         "05KD003", "05KE010", "05KH007", "05KJ001", "05LC001", "05LH005", "05LJ025", "05LL002", "05LL014", "05LM001",
         "05LM006", "05MD004", "05MD005", "05ME001", "05MF001", "05MH001", "05MH005", "05MJ001", "05NA003", "05NB001",
         "05NB021", "05NB036", "05ND010", "05NG001", "05NG024", "05OB007", "05OB016", "05OB021", "05OC001", "05OC012",
         "05OG001", "05PA006", "05PB014", "05PB018", "05PG001", "05PH003", "05QA004", "05QC001", "05QD006", "05QE008",
         "05QE009", "05QE012", "05RA001", "05RB003", "05RC001", "05TD001", "05TE002", "05TF002", "05TG001", "05TG002",
         "05TG003", "05TG006", "05UA003", "05UF004", "05UG001", "05UH001", "05UH002", "06AA002", "06AB001", "06AB002",
         "06AD006", "06AF001", "06AG002", "06BA002", "06BB003", "06BB005", "06BD001", "06CB002", "06CD002", "06DA002",
         "06DA004", "06DA005", "06DC001", "06DD002", "06DD003", "06EA002", "06EA006", "06EA011", "06FA001", "06FB001",
         "06FB002", "06FC001", "06FD001", "06FD002"]

station_us = ["05017500", "05020500", "05030500", "05046000", "05046475", "05050000", "05051300", "05051500", "05051522", 
         "05051600", "05052000", "05052500", "05053000", "05054000", "05054500", "05055300", "05055400", "05056000", 
         "05057000", "05057200", "05058000", "05058500", "05058700", "05059000", "05059500", "05059700", "05059715", 
         "05060000", "05060100", "05060500", "05061000", "05061500", "05062000", "05062500", "05063398", "05064000", 
         "05064500", "05065500", "05066500", "05067500", "05069000", "05070000", "05074500", "05075000", "05076000", 
         "05078000", "05078230", "05078500", "05079000", "05080000", "05082500", "05082625", "05084000", "05085000", 
         "05085450", "05087500", "05090000", "05092000", "05094000", "05099400", "05099600", "05100000", "05101000", 
         "05104500", "05106000", "05107500", "05112000", "05113600", "05114000", "05116000", "05116500", "05120000", 
         "05120500", "05122000", "05123400", "05123510", "05124000", "05124480", "05125000", "05126210", "05127500", 
         "05129115", "05129515", "05131500", "05132000", "05133500", "05134200"]


# Define date range
start_date = "1980-01-01"
end_date = "2018-01-10"

# Fetch data for the given stations and date range
combined_data_ca, station_info_ca = fetch_hydrometric_data_ca(station_ca, start_date, end_date)
combined_data_us, station_info_us = extract_flow_data_us(station_us, start_date, end_date)

# Combine data into a single DataFrame
combined_data = pd.merge(combined_data_ca, combined_data_us, on='Date', how='outer')

# Combine station info
combined_station_info = station_info_ca + station_info_us

# Write the data to a file
write_flow_data_to_file_obstxt('output.txt', combined_data, combined_station_info)
write_flow_data_to_file_ensim('output_ensim.txt', combined_data, combined_station_info)

# Print the resulting DataFrame and station information
print(combined_data.head())
print(combined_station_info)

In [ ]:
# Example usage:
# Define sample lists of station IDs
station_ca = ["05GG001", "05AC012"]
station_us = ["05017500", "05020500"]

# Define date range
start_date = "1980-01-01"
end_date = "2018-01-10"

# Fetch data for the given stations and date range
combined_data_ca, station_info_ca = fetch_hydrometric_data_ca(station_ca, start_date, end_date)
combined_data_us, station_info_us = extract_flow_data_us(station_us, start_date, end_date)

# Combine data into a single DataFrame
combined_data = pd.merge(combined_data_ca, combined_data_us, on='Date', how='outer')

# Combine station info
combined_station_info = station_info_ca + station_info_us

# Write the data to a file
write_flow_data_to_file_obstxt('output.txt', combined_data, combined_station_info)
write_flow_data_to_file_ensim('output_ensim.txt', combined_data, combined_station_info)

# Print the resulting DataFrame and station information
print(combined_data.head())
print(combined_station_info)

In [ ]:
station_ca = ["05AA004", "05AA008", "05AA011", "05AA022", "05AA024", "05AA027", "05AA028", "05AA035", "05AB005", "05AB021",
         "05AB029", "05AB041", "05AC003", "05AC012", "05AC023", "05AC030", "05AC031", "05AD003", "05AD005", "05AD007",
         "05AD010", "05AD028", "05AD035", "05AD041", "05AE002", "05AE005", "05AE006", "05AE016", "05AE027", "05AE043",
         "05AG003", "05AG006", "05AH005", "05AH037", "05AH041", "05AH052", "05AH053", "05AJ001", "05BA001", "05BA002",
         "05BB001", "05BC001", "05BG006", "05BG010", "05BH004", "05BH005", "05BH014", "05BH015", "05BH016", "05BJ001",
         "05BJ004", "05BJ010", "05BK001", "05BL004", "05BL007", "05BL012", "05BL013", "05BL014", "05BL019", "05BL022",
         "05BL023", "05BL024", "05BL027", "05BM002", "05BM004", "05BM007", "05BM008", "05BM014", "05BN002", "05BN006",
         "05BN012", "05CA002", "05CA004", "05CA009", "05CA011", "05CA012", "05CB001", "05CB002", "05CB004", "05CB007",
         "05CC001", "05CC002", "05CC007", "05CC008", "05CC009", "05CC011", "05CC013", "05CD006", "05CD007", "05CE001",
         "05CE002", "05CE005", "05CE006", "05CE007", "05CE012", "05CE018", "05CE020", "05CG004", "05CG006", "05CH007",
         "05CH008", "05CH011", "05CJ006", "05CJ012", "05CK001", "05CK004", "05CK005", "05CK006", "05DA009", "05DB002",
         "05DB006", "05DC001", "05DC006", "05DD009", "05DF001", "05EA005", "05EF001", "05FA001", "05FA011", "05FE004",
         "05GG001", "05HD036", "05HD039", "05HG001", "05JE006", "05JF001", "05JF005", "05JK002", "05JK007", "05JM001",
         "05KD003", "05KE010", "05KH007", "05KJ001", "05LC001", "05LH005", "05LJ025", "05LL002", "05LL014", "05LM001",
         "05LM006", "05MD004", "05MD005", "05ME001", "05MF001", "05MH001", "05MH005", "05MJ001", "05NA003", "05NB001",
         "05NB021", "05NB036", "05ND010", "05NG001", "05NG024", "05OB007", "05OB016", "05OB021", "05OC001", "05OC012",
         "05OG001", "05PA006", "05PB014", "05PB018", "05PG001", "05PH003", "05QA004", "05QC001", "05QD006", "05QE008",
         "05QE009", "05QE012", "05RA001", "05RB003", "05RC001", "05TD001", "05TE002", "05TF002", "05TG001", "05TG002",
         "05TG003", "05TG006", "05UA003", "05UF004", "05UG001", "05UH001", "05UH002", "06AA002", "06AB001", "06AB002",
         "06AD006", "06AF001", "06AG002", "06BA002", "06BB003", "06BB005", "06BD001", "06CB002", "06CD002", "06DA002",
         "06DA004", "06DA005", "06DC001", "06DD002", "06DD003", "06EA002", "06EA006", "06EA011", "06FA001", "06FB001",
         "06FB002", "06FC001", "06FD001", "06FD002"]

station_us = ["05017500", "05020500", "05030500", "05046000", "05046475", "05050000", "05051300", "05051500", "05051522", 
         "05051600", "05052000", "05052500", "05053000", "05054000", "05054500", "05055300", "05055400", "05056000", 
         "05057000", "05057200", "05058000", "05058500", "05058700", "05059000", "05059500", "05059700", "05059715", 
         "05060000", "05060100", "05060500", "05061000", "05061500", "05062000", "05062500", "05063398", "05064000", 
         "05064500", "05065500", "05066500", "05067500", "05069000", "05070000", "05074500", "05075000", "05076000", 
         "05078000", "05078230", "05078500", "05079000", "05080000", "05082500", "05082625", "05084000", "05085000", 
         "05085450", "05087500", "05090000", "05092000", "05094000", "05099400", "05099600", "05100000", "05101000", 
         "05104500", "05106000", "05107500", "05112000", "05113600", "05114000", "05116000", "05116500", "05120000", 
         "05120500", "05122000", "05123400", "05123510", "05124000", "05124480", "05125000", "05126210", "05127500", 
         "05129115", "05129515", "05131500", "05132000", "05133500", "05134200"]


# Define date range
start_date = "1980-01-01"
end_date = "2018-01-10"

# Fetch data for the given stations and date range
combined_data_ca, station_info_ca = fetch_hydrometric_data_ca(station_ca, start_date, end_date)
combined_data_us, station_info_us = extract_flow_data_us(station_us, start_date, end_date)

# Combine data into a single DataFrame
combined_data = pd.merge(combined_data_ca, combined_data_us, on='Date', how='outer')

# Combine station info
combined_station_info = station_info_ca + station_info_us

# Write the data to a file
write_flow_data_to_file_obstxt('output.txt', combined_data, combined_station_info)
write_flow_data_to_file_ensim('output_ensim.txt', combined_data, combined_station_info)

# Print the resulting DataFrame and station information
print(combined_data.head())
print(combined_station_info)

In [3]:
import pandas as pd
import numpy as np
import requests
from owslib.ogcapi.features import Features
from datetime import datetime, timedelta

# Initialize the Features API
oafeat = Features("https://api.weather.gc.ca/")

def create_date_range(start_date, end_date):
    return pd.date_range(start=start_date, end=end_date)

def extract_flow_data_us(station_list, start_date, end_date):
    dates = create_date_range(start_date, end_date)
    data_dict = {'Date': dates}
    date_index_dict = {str(date.date()): idx for idx, date in enumerate(dates)}
    station_info = []

    for station in station_list:
        data_dict[station] = [np.nan] * len(dates)
    
    for station in station_list:
        url = f"https://waterservices.usgs.gov/nwis/dv/?format=json&sites={station}&startDT={start_date}&endDT={end_date}&parameterCd=00060&statCd=00003"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if 'timeSeries' in data['value']:
                records = data['value']['timeSeries'][0]['values'][0]['value']
                flow_data = pd.DataFrame(records)
                flow_data['value'] = pd.to_numeric(flow_data['value'], errors='coerce')
                flow_data['dateTime'] = pd.to_datetime(flow_data['dateTime']).dt.date.astype(str)
                for date, flow in zip(flow_data['dateTime'], flow_data['value']):
                    if date in date_index_dict:
                        date_index = date_index_dict[date]
                        data_dict[station][date_index] = flow
                
                # Collect station information
                site_info = data['value']['timeSeries'][0]['sourceInfo']
                station_info.append({
                    'Station_Number': site_info['siteCode'][0]['value'],
                    'Station_Name': site_info['siteName'],
                    'Latitude': site_info['geoLocation']['geogLocation']['latitude'],
                    'Longitude': site_info['geoLocation']['geogLocation']['longitude'],
                    'Drainage_Area': next((prop['value'] for prop in site_info['siteProperty'] if prop['name'] == 'drain_area_va'), None)
                })
            else:
                print(f"Flow data column not found for station: {station}")
        else:
            print(f"Failed to retrieve data for station: {station}")
    
    combined_df = pd.DataFrame(data_dict)
    return combined_df, station_info

def fetch_hydrometric_data_ca(station_numbers, start_date, end_date, limit=500):
    dates = create_date_range(start_date, end_date)
    data_dict = {'Date': dates}
    date_index_dict = {str(date.date()): idx for idx, date in enumerate(dates)}
    station_info = []

    for station_number in station_numbers:
        data_dict[station_number] = [np.nan] * len(dates)

        offset = 0
        full_data = []

        while True:
            url = f"https://api.weather.gc.ca/collections/hydrometric-daily-mean/items"
            params = {
                'STATION_NUMBER': station_number,
                'datetime': f"{start_date}/{end_date}",
                'limit': limit,
                'offset': offset,
                'f': 'json'
            }

            response = requests.get(url, params=params)
            response_data = response.json()

            if 'features' in response_data and response_data['features']:
                full_data.extend(response_data['features'])
                offset += limit
                if len(response_data['features']) < limit:
                    break
            else:
                break

        if full_data:
            data_list = [
                {
                    'Date': feature['properties']['DATE'],
                    'value': feature['properties']['DISCHARGE'] if feature['properties']['DISCHARGE'] is not None else -1000
                }
                for feature in full_data
            ]

            flow_data = pd.DataFrame(data_list)
            flow_data['value'] = pd.to_numeric(flow_data['value'], errors='coerce')
            flow_data['Date'] = pd.to_datetime(flow_data['Date']).dt.date.astype(str)

            for date, flow in zip(flow_data['Date'], flow_data['value']):
                if date in date_index_dict:
                    date_index = date_index_dict[date]
                    data_dict[station_number][date_index] = flow

            first_feature = full_data[0]['properties']
            geometry = full_data[0]['geometry']
            station_info.append({
                'Station_Number': first_feature['STATION_NUMBER'],
                'Station_Name': first_feature['STATION_NAME'],
                'Latitude': geometry['coordinates'][1],
                'Longitude': geometry['coordinates'][0],
                'Drainage_Area': first_feature.get('DRAINAGE_AREA_GROSS', None)
            })
        else:
            print(f"Flow data not found for station: {station_number}")

    combined_df = pd.DataFrame(data_dict)
    return combined_df, station_info

def write_flow_data_to_file_obstxt(file_path, flow_data, site_details):
    flow_data = flow_data.fillna(-1.000)
    flow_data['Date'] = pd.to_datetime(flow_data['Date'])

    with open(file_path, "w") as file_conn:
        start_date = flow_data['Date'].min()
        end_date = flow_data['Date'].max()
        file_conn.write(f"Observedstreamflow\t{start_date.strftime('%Y/%m/%d')}\t{end_date.strftime('%Y/%m/%d')}\n")
        num_stations = flow_data.shape[1] - 1
        num_days = flow_data.shape[0]
        start_year = start_date.strftime('%Y')
        start_day_of_year = start_date.timetuple().tm_yday
        file_conn.write(f"{num_stations}  {num_days}  {num_days}  24 {start_year}  {start_day_of_year} 00\n")
        
        for station_id in flow_data.columns[1:]:
            station_info = next((item for item in site_details if item["Station_Number"] == station_id), None)
            if station_info:
                lat = station_info['Latitude']
                lon = station_info['Longitude']
                drainage_area = station_info['Drainage_Area']
                if drainage_area is None:
                    drainage_area = -1.0
                station_name = station_info['Station_Name']
                file_conn.write(f"{int(lat * 60):4d} {int(lon * 60):4d} {lat:12.6f} {lon:12.6f} {station_id:12s} {float(drainage_area):12.3f} {station_name}\n")
        
        for i in range(num_days):
            flow_values = flow_data.iloc[i, 1:].values
            formatted_flow_values = " ".join(f"{x:12.4f}" for x in flow_values)
            file_conn.write(f"{formatted_flow_values}\n")

def write_flow_data_to_file_ensim(file_path, flow_data, site_details):
    flow_data = flow_data.fillna(-1.000)

    header = [
        "########################################",
        ":FileType tb0  ASCII  EnSim 1.0",
        "#",
        "# DataType               Time Series",
        "#",
        ":Application             EnSimHydrologic",
        ":Version                 2.1.23",
        ":WrittenBy          PythonScript",
        f":CreationDate       {datetime.now().strftime('%Y-%m-%d')}",
        "#",
        "#---------------------------------------",
        ":SourceFile                   flow_data",
        "#",
        ":Name               streamflow",
        "#",
        ":Projection         LATLONG",
        ":Ellipsoid          WGS84",
        "#",
        f":StartTime          {flow_data['Date'].iloc[0]} 00:00:00.00000",
        "#",
        ":AttributeUnits            1.0000000",
        ":DeltaT               24",
        ":RoutingDeltaT         1",
        "#",
        ":ColumnMetaData",
        f"   :ColumnUnits             {' '.join(['m3/s' for _ in range(flow_data.shape[1] - 1)])}",
        f"   :ColumnType             {' '.join(['float' for _ in range(flow_data.shape[1] - 1)])}",
        f"   :ColumnName           {' '.join(flow_data.columns[1:])}",
        "   :ColumnLocationX    " + ' '.join([f"{site['Longitude'] * 60:.6f}" for site in site_details]),
        "   :ColumnLocationY    " + ' '.join([f"{site['Latitude'] * 60:.6f}" for site in site_details]),
        f"   :coeff1            {' '.join(['0.0000E+00' for _ in range(flow_data.shape[1] - 1)])}",
        f"   :coeff2            {' '.join(['0.0000E+00' for _ in range(flow_data.shape[1] - 1)])}",
        f"   :coeff3            {' '.join(['0.0000E+00' for _ in range(flow_data.shape[1] - 1)])}",
        f"   :coeff4            {' '.join(['0.0000E+00' for _ in range(flow_data.shape[1] - 1)])}",
        f"   :Value1            {' '.join(['1' for _ in range(flow_data.shape[1] - 1)])}",
        ":EndColumnMetaData",
        ":endHeader"
    ]
    
    with open(file_path, "w") as file_conn:
        file_conn.write("\n".join(header) + "\n")
        
        for _, row in flow_data.iterrows():
            flows = row[1:].values
            flow_string = " ".join(f"{flow:12.4f}" for flow in flows)
            file_conn.write(f"{' ' * 22}{flow_string}\n")

station_ca = ["05AA004", "05AA008", "05AA011", "05AA022", "05AA024", "05AA027", "05AA028", "05AA035", "05AB005", "05AB021",
         "05AB029", "05AB041", "05AC003", "05AC012", "05AC023", "05AC030", "05AC031", "05AD003", "05AD005", "05AD007",
         "05AD010", "05AD028", "05AD035", "05AD041", "05AE002", "05AE005", "05AE006", "05AE016", "05AE027", "05AE043",
         "05AG003", "05AG006", "05AH005", "05AH037", "05AH041", "05AH052", "05AH053", "05AJ001", "05BA001", "05BA002",
         "05BB001", "05BC001", "05BG006", "05BG010", "05BH004", "05BH005", "05BH014", "05BH015", "05BH016", "05BJ001",
         "05BJ004", "05BJ010", "05BK001", "05BL004", "05BL007", "05BL012", "05BL013", "05BL014", "05BL019", "05BL022",
         "05BL023", "05BL024", "05BL027", "05BM002", "05BM004", "05BM007", "05BM008", "05BM014", "05BN002", "05BN006",
         "05BN012", "05CA002", "05CA004", "05CA009", "05CA011", "05CA012", "05CB001", "05CB002", "05CB004", "05CB007",
         "05CC001", "05CC002", "05CC007", "05CC008", "05CC009", "05CC011", "05CC013", "05CD006", "05CD007", "05CE001",
         "05CE002", "05CE005", "05CE006", "05CE007", "05CE012", "05CE018", "05CE020", "05CG004", "05CG006", "05CH007",
         "05CH008", "05CH011", "05CJ006", "05CJ012", "05CK001", "05CK004", "05CK005", "05CK006", "05DA009", "05DB002",
         "05DB006", "05DC001", "05DC006", "05DD009", "05DF001", "05EA005", "05EF001", "05FA001", "05FA011", "05FE004",
         "05GG001", "05HD036", "05HD039", "05HG001", "05JE006", "05JF001", "05JF005", "05JK002", "05JK007", "05JM001",
         "05KD003", "05KE010", "05KH007", "05KJ001", "05LC001", "05LH005", "05LJ025", "05LL002", "05LL014", "05LM001",
         "05LM006", "05MD004", "05MD005", "05ME001", "05MF001", "05MH001", "05MH005", "05MJ001", "05NA003", "05NB001",
         "05NB021", "05NB036", "05ND010", "05NG001", "05NG024", "05OB007", "05OB016", "05OB021", "05OC001", "05OC012",
         "05OG001", "05PA006", "05PB014", "05PB018", "05PG001", "05PH003", "05QA004", "05QC001", "05QD006", "05QE008",
         "05QE009", "05QE012", "05RA001", "05RB003", "05RC001", "05TD001", "05TE002", "05TF002", "05TG001", "05TG002",
         "05TG003", "05TG006", "05UA003", "05UF004", "05UG001", "05UH001", "05UH002", "06AA002", "06AB001", "06AB002",
         "06AD006", "06AF001", "06AG002", "06BA002", "06BB003", "06BB005", "06BD001", "06CB002", "06CD002", "06DA002",
         "06DA004", "06DA005", "06DC001", "06DD002", "06DD003", "06EA002", "06EA006", "06EA011", "06FA001", "06FB001",
         "06FB002", "06FC001", "06FD001", "06FD002"]

station_us = ["05017500", "05020500", "05030500", "05046000", "05046475", "05050000", "05051300", "05051500", "05051522", 
         "05051600", "05052000", "05052500", "05053000", "05054000", "05054500", "05055300", "05055400", "05056000", 
         "05057000", "05057200", "05058000", "05058500", "05058700", "05059000", "05059500", "05059700", "05059715", 
         "05060000", "05060100", "05060500", "05061000", "05061500", "05062000", "05062500", "05063398", "05064000", 
         "05064500", "05065500", "05066500", "05067500", "05069000", "05070000", "05074500", "05075000", "05076000", 
         "05078000", "05078230", "05078500", "05079000", "05080000", "05082500", "05082625", "05084000", "05085000", 
         "05085450", "05087500", "05090000", "05092000", "05094000", "05099400", "05099600", "05100000", "05101000", 
         "05104500", "05106000", "05107500", "05112000", "05113600", "05114000", "05116000", "05116500", "05120000", 
         "05120500", "05122000", "05123400", "05123510", "05124000", "05124480", "05125000", "05126210", "05127500", 
         "05129115", "05129515", "05131500", "05132000", "05133500", "05134200"]

start_date = "1980-01-01"
end_date = "2018-01-10"

combined_data_ca, station_info_ca = fetch_hydrometric_data_ca(station_ca, start_date, end_date)
combined_data_us, station_info_us = extract_flow_data_us(station_us, start_date, end_date)

combined_data = pd.merge(combined_data_ca, combined_data_us, on='Date', how='outer')

combined_station_info = station_info_ca + station_info_us

write_flow_data_to_file_obstxt('output.txt', combined_data, combined_station_info)
write_flow_data_to_file_ensim('output_ensim.txt', combined_data, combined_station_info)

print(combined_data.head())
print(combined_station_info)


        Date  05AA004  05AA008  05AA011  05AA022  05AA024  05AA027  05AA028  \
0 1980-01-01      NaN     1.01      NaN     2.07     3.75      NaN      NaN   
1 1980-01-02      NaN     1.04      NaN     2.05     3.62      NaN      NaN   
2 1980-01-03      NaN     1.04      NaN     2.04     3.50      NaN      NaN   
3 1980-01-04      NaN     1.03      NaN     2.01     3.40      NaN      NaN   
4 1980-01-05      NaN     1.02      NaN     1.98     3.28      NaN      NaN   

   05AA035  05AB005  ...  05124480  05125000  05126210  05127500  05129115  \
0      NaN      NaN  ...      73.0       NaN       NaN     703.0     239.0   
1      NaN      NaN  ...      72.0       NaN       NaN     703.0     235.0   
2      NaN      NaN  ...      72.0       NaN       NaN     703.0     230.0   
3      NaN      NaN  ...      70.0       NaN       NaN     703.0     226.0   
4      NaN      NaN  ...      70.0       NaN       NaN     703.0     221.0   

   05129515  05131500  05132000  05133500  05134200  
0 